### Basic imports and definition of variables

In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np
import pathlib
import datetime 

input_path = "dataset/raw"
output_path ="dataset/preprocessed/"

look_back_interval=10
all_weather = False
save_intermediary_files = True

output = pathlib.Path(output_path)
output.mkdir(parents=True, exist_ok=True)
input = pathlib.Path(input_path)

kpi_file= input/"rl-kpis.tsv"
distances_file = input/"distances.tsv"
met_forecast_file = input/"met-forecast.tsv"
rl_sites_file = input/"rl-sites.tsv"
met_stations_file = input/"met-stations.tsv"
met_real_file = input/"met-real.tsv"

other = ["precipitation_min", "precipitation_max", "precipitation_std", "precipitation_mean"]

removable_features = ["unavail_second", "severaly_error_second", "error_second", "bbe", "avail_time", "mw_connection_no", "precipitation_coeff_max"]

ordinal_enconder = OrdinalEncoder(categories=[['clear sky', 'hot day', 'scattered clouds', 'few clouds', 'overcast clouds','foggy', 'windy', 'misty', 'light rain', 
                                              'light rain showers', 'light intensity shower rain', 'light snow','snow', 'sleet',
                                                'rain', 'heavy rain showers','heavy rain', 
                                              'thunderstorm with heavy rain', 'heavy thunderstorm with rain showers']],
                                  
                                  handle_unknown='use_encoded_value',
                                  unknown_value=np.nan
                                  )

one_hot_encoder = OneHotEncoder(handle_unknown='error')


### Function to find K closest stations

In [2]:
def finding_closest_stations(rl_sites_file, distances_file, met_stations_file, k=3):
    rl_sites_df = pd.read_csv(rl_sites_file, sep="\t", index_col=0)
    distances_df = pd.read_csv(distances_file, sep="\t", index_col=0)
    met_stations_df = pd.read_csv(met_stations_file, sep="\t", index_col=0)

    rl_stations = rl_sites_df["site_id"].unique()
    met_stations = met_stations_df["station_no"].unique()

    distances_df = distances_df.loc[met_stations, rl_stations]

    closest_stations = dict()
    for rl_station in rl_stations:
        closest_stations[rl_station] = set(distances_df.nsmallest(k, [rl_station]).index.tolist())

    return closest_stations

#select most frequent option
def select_frequent(series):
    frequent = pd.Series.mode(series)
    if len(frequent) > 1:
        return frequent[1]
    elif len(frequent) > 0:
        return frequent[0]
    
    return np.NaN

def timeseries_processing(time_sentitive_dataset, time_sensitive_features, identifiers, labels, output, df_type = "train", full=True):
    ordered_features = []
    for i in range(1-look_back_interval, 0, 1):
        for feature in time_sensitive_features:
            ordered_features.append(f"T{i}_{feature}")
    
    ordered_features += time_sensitive_features
    if full:
        for feature in time_sensitive_features:
            historical_sen_dataset = time_sentitive_dataset.loc[:, identifiers]
            for i in range(-1,  -look_back_interval, -1):
                historical_sen_dataset[f"T{i}"] = historical_sen_dataset["datetime"] + pd.DateOffset(days=i)

            feature_view = time_sentitive_dataset[identifiers + [feature]]
            for i in range(-1,  -look_back_interval, -1):
                target_day_column_name = f"T{i}"

                historical_sen_dataset = historical_sen_dataset.merge(feature_view, 
                        how = "left", 
                        left_on = ("site_id", "mlid", target_day_column_name),
                        right_on = identifiers,
                        suffixes = ("", "_y")
                )
                historical_sen_dataset.rename(columns={ feature: f"{target_day_column_name}_{feature}"}, inplace=True)

            historical_sen_dataset.drop(columns=["datetime_y"], inplace=True)

            historical_sen_dataset.drop(columns=[f"T{i}" for i in range(-1,  -look_back_interval, -1)], inplace=True)

            time_sentitive_dataset = time_sentitive_dataset.merge(historical_sen_dataset, 
                        how="left", 
                        on=["datetime", "site_id", "mlid"])

        time_sentitive_dataset = time_sentitive_dataset.dropna()
        time_sentitive_dataset[labels] = time_sentitive_dataset[labels].astype(int)

        time_sentitive_dataset = time_sentitive_dataset[identifiers + ordered_features + labels]
        if df_type != "train":
            if len(time_sentitive_dataset) > 1:
                np.savetxt(output/f"x_{df_type}.csv", time_sentitive_dataset[ordered_features].values, delimiter=",", fmt="%5.2f")
                np.savetxt(output/f"y_{df_type}.csv", time_sentitive_dataset[["1-day-predict"]].values, delimiter=",", fmt="%d")
        else:
            rl_mlid_combos =  (time_sentitive_dataset["site_id"] + "%" + time_sentitive_dataset["mlid"]).unique()

            for rl_mlid in rl_mlid_combos:
                site_id, mlid = rl_mlid.split("%")
                rl_mlid_df = time_sentitive_dataset.loc[(time_sentitive_dataset["site_id"] == site_id) & (time_sentitive_dataset["mlid"] == mlid)]
                if len(rl_mlid_df) > 1:
                        site_folder = output / site_id
                        site_folder.mkdir(parents=True, exist_ok=True)
                        rl_mlid_df.to_csv(site_folder/f"{mlid}_time_sensitive_features.csv", index=None)
    else:
        if df_type != "train":
            for feature in time_sensitive_features:
                historical_sen_dataset = time_sentitive_dataset.loc[:, identifiers]
                for i in range(-1,  -look_back_interval, -1):
                    historical_sen_dataset[f"T{i}"] = historical_sen_dataset["datetime"] + pd.DateOffset(days=i)

                feature_view = time_sentitive_dataset[identifiers + [feature]]
                for i in range(-1,  -look_back_interval, -1):
                    target_day_column_name = f"T{i}"

                    historical_sen_dataset = historical_sen_dataset.merge(feature_view, 
                            how = "left", 
                            left_on = ("site_id", "mlid", target_day_column_name),
                            right_on = identifiers,
                            suffixes = ("", "_y")
                    )
                    historical_sen_dataset.rename(columns={ feature: f"{target_day_column_name}_{feature}"}, inplace=True)

                historical_sen_dataset.drop(columns=["datetime_y"], inplace=True)

                historical_sen_dataset.drop(columns=[f"T{i}" for i in range(-1,  -look_back_interval, -1)], inplace=True)

                time_sentitive_dataset = time_sentitive_dataset.merge(historical_sen_dataset, 
                            how="left", 
                            on=["datetime", "site_id", "mlid"])

            time_sentitive_dataset = time_sentitive_dataset.dropna()
            time_sentitive_dataset[labels] = time_sentitive_dataset[labels].astype(int)

            time_sentitive_dataset = time_sentitive_dataset[identifiers + ordered_features + labels]
            if len(time_sentitive_dataset) > 1:
                np.savetxt(output/f"x_{df_type}.csv", time_sentitive_dataset[ordered_features].values, delimiter=",", fmt="%5.2f")
                np.savetxt(output/f"y_{df_type}.csv", time_sentitive_dataset[["1-day-predict"]].values, delimiter=",", fmt="%d")
        else:
            output = output/"train_data"
            output.mkdir(parents=True, exist_ok=True)
            rl_mlid_combos =  (time_sentitive_dataset["site_id"] + "%" + time_sentitive_dataset["mlid"]).unique()

            for rl_mlid in rl_mlid_combos:
                site_id, mlid = rl_mlid.split("%")
                rl_mlid_df = time_sentitive_dataset.loc[(time_sentitive_dataset["site_id"] == site_id) & (time_sentitive_dataset["mlid"] == mlid)]

                for feature in time_sensitive_features:
                    historical_sen_dataset = rl_mlid_df.loc[:, identifiers]
                    for i in range(-1,  -look_back_interval, -1):
                        historical_sen_dataset[f"T{i}"] = historical_sen_dataset["datetime"] + pd.DateOffset(days=i)

                    feature_view = rl_mlid_df[identifiers + [feature]]
                    for i in range(-1,  -look_back_interval, -1):
                        target_day_column_name = f"T{i}"

                        historical_sen_dataset = historical_sen_dataset.merge(feature_view, 
                                how = "left", 
                                left_on = ("site_id", "mlid", target_day_column_name),
                                right_on = identifiers,
                                suffixes = ("", "_y")
                        )
                        historical_sen_dataset.rename(columns={ feature: f"{target_day_column_name}_{feature}"}, inplace=True)

                    historical_sen_dataset.drop(columns=["datetime_y"], inplace=True)

                    historical_sen_dataset.drop(columns=[f"T{i}" for i in range(-1,  -look_back_interval, -1)], inplace=True)

                    rl_mlid_df = rl_mlid_df.merge(historical_sen_dataset, 
                                how="left", 
                                on=["datetime", "site_id", "mlid"])

                rl_mlid_df = rl_mlid_df.dropna()
                rl_mlid_df[labels] = rl_mlid_df[labels].astype(int)

                rl_mlid_df = rl_mlid_df[identifiers + ordered_features + labels]
                
                if len(rl_mlid_df) > 1:
                    site_folder = output / site_id
                    site_folder.mkdir(parents=True, exist_ok=True)
                    rl_mlid_df.to_csv(site_folder/f"{mlid}_time_sensitive_features.csv", index=None)

### Preprocessing weather features

In [3]:
# Loading forecasting data
met_forecast_df = pd.read_csv(met_forecast_file, sep="\t", index_col=0)

met_forecast_df["datetime"] = pd.to_datetime(met_forecast_df["datetime"])

# Filtering the reports to only include the morning report, and removing that column afterwards. (usually there are morning and afternoon reports)
met_forecast_df = met_forecast_df.groupby(by=["station_no", "datetime"], group_keys=False).agg(select_frequent)
met_forecast_df.reset_index(level=["station_no", "datetime"], inplace=True)
met_forecast_df.drop(columns=["report_time"], inplace=True)

#Filtering between 5 days of forecast or just day 1
if not all_weather:
    columns = [column for column in met_forecast_df.columns if "day1" not in column and column not in ["station_no", "datetime"]]
    met_forecast_df.drop(columns=columns, inplace=True)

## Transforming weather features in ordinal encoding vectors
if all_weather:
    weather_features = [f"weather_day{i}" for i in range(1,6)]
else:
    weather_features = ["weather_day1"]

for weather_feature in weather_features:
    met_forecast_df[weather_feature] = ordinal_enconder.fit_transform(met_forecast_df[weather_feature].to_numpy().reshape(-1,1).tolist()) 

# Loading the real data and removing the hour of when it was collected
met_real_df = pd.read_csv(met_real_file, sep="\t", index_col=0)
met_real_df.drop(columns=["datetime", "measured_hour"], inplace=True)

met_real_df["measured_date"] = pd.to_datetime(met_real_df["measured_date"])
met_real_df.rename(columns={"measured_date":"datetime"}, inplace=True)


### Loading KPI file & finding the closest stations

In [4]:
#Get the closest met station to each radio link one.
closest_stations = finding_closest_stations(rl_sites_file, distances_file, met_stations_file)

#List of columns that uniquelly identify an entry in the kpi_df
identifiers = ["site_id", "mlid", "datetime"]

kpi_df = pd.read_csv(kpi_file, sep="\t", index_col=0)

#Transform the datetime column to the correct format
kpi_df["datetime"] = pd.to_datetime(kpi_df["datetime"])

### Aggregating real and forecast weather for each site_id

In [5]:
met_real_agg_df = None
#Adding closest station to each entry according to the id of the radio link
for site_id in kpi_df["site_id"].unique():
    temp_df = met_real_df[met_real_df["station_no"].isin(closest_stations[site_id])]\
                    .drop(columns=["station_no"])   \
                    .groupby(by=["datetime"], group_keys=False).agg(['min', 'max', 'mean', 'std'])
    
    temp_df.reset_index(level=["datetime"], inplace=True)
    temp_df["site_id"] = site_id

    if type(met_real_agg_df) == pd.DataFrame:
        met_real_agg_df = pd.concat([met_real_agg_df, temp_df], ignore_index=True)
    else:
        met_real_agg_df = temp_df

#Transforming multi-level columns to single level
met_real_agg_df.columns = ["_".join(x)  if x[1] != '' else x[0] for x in met_real_agg_df.columns]

#Mergin k closest stations to each entry according to the id of the radio link

met_forecast_agg_df = None

for site_id in kpi_df["site_id"].unique():
    filtered_df = met_forecast_df[met_forecast_df["station_no"].isin(closest_stations[site_id])]\
                    .drop(columns=["station_no"])
                    
    temp_df = filtered_df.drop(columns=["weather_day1"]).groupby(by=["datetime"], group_keys=False).mean()

    temp_df["weather_day1"] = filtered_df[["datetime", "weather_day1"]].groupby(by=["datetime"], group_keys=False) \
                    .agg(select_frequent)["weather_day1"]    
    
    temp_df.reset_index(level=["datetime"], inplace=True)
    temp_df["site_id"] = site_id

    if type(met_forecast_agg_df) == pd.DataFrame:
        met_forecast_agg_df = pd.concat([met_forecast_agg_df, temp_df], ignore_index=True)
    else:
        met_forecast_agg_df = temp_df

### Merging KPI df with met forecast and real

In [6]:
## Merging kpis with forecast data
kpi_df = kpi_df.merge(met_forecast_agg_df, on=["datetime", "site_id"])
kpi_df = kpi_df.merge(met_real_agg_df, on=["datetime", "site_id"])

#Remove unnecessary columns
kpi_df.drop(columns=removable_features, inplace=True)

if "scalibility_score" in kpi_df.columns:
    kpi_df.drop(columns=["scalibility_score"], inplace=True)

## Create the labels for each entry 1-day after

In [7]:
df_labels = kpi_df.loc[:, identifiers]

df_labels["T+1"] = df_labels["datetime"] + pd.DateOffset(days=1)

df_labels_view = kpi_df[identifiers + ["rlf"]]

df_labels = df_labels.merge(df_labels_view, 
            how = "left", 
            left_on = ("site_id", "mlid", "T+1"),
            right_on = identifiers,
            suffixes = ("", "_y")
)
df_labels.rename(columns={"rlf": "T+1_rlf"}, inplace=True)

df_labels.drop(columns=["datetime_y"], inplace=True)

df_labels["1-day-predict"] = df_labels["T+1_rlf"]

df_labels = df_labels[["datetime", "site_id", "mlid", "1-day-predict"]]

kpi_df = kpi_df.merge(df_labels, 
                        how="left", 
                        on=["datetime", "site_id", "mlid"])


static_features = ["card_type", "freq_band", "type", "tip", "adaptive_modulation", "freq_band", "modulation"]
labels = ["rlf", "1-day-predict"]

time_sensitive_features = [feature for feature in kpi_df.columns if feature not in static_features and feature not in labels and feature not in identifiers]

## Seperating static / time dependent features and creating the static dataset 

In [8]:
time_sentitive_dataset = kpi_df.loc[:, identifiers + time_sensitive_features + labels]

static_dataset = kpi_df.loc[:, identifiers + static_features + labels]
static_dataset = static_dataset.dropna()

one_hot_encoder.fit(static_dataset[static_features])

static_dataset = pd.concat([static_dataset,
                            pd.DataFrame(one_hot_encoder.transform(static_dataset[static_features]).toarray(),
                                                         columns=one_hot_encoder.get_feature_names_out())
                            ],
                        axis=1
                        )

static_dataset.drop(columns=static_features, inplace=True)

if save_intermediary_files:
    static_dataset.to_csv(output/"preprocessed_static_features.csv", index=None)
    time_sentitive_dataset.to_csv(output/"preprocessed_timeseries.csv", index=None)
    

## Dividing between train/cv/test

In [9]:
#print(time_sentitive_dataset)
train_df = time_sentitive_dataset.loc[time_sentitive_dataset["datetime"] < datetime.datetime(2019, 9,1)]

cv_df = time_sentitive_dataset.loc[(time_sentitive_dataset["datetime"] >= datetime.datetime(2019, 9,1)) \
                                   & (time_sentitive_dataset["datetime"] < datetime.datetime(2020, 1,1) )]

test_df = time_sentitive_dataset.loc[(time_sentitive_dataset["datetime"] >= datetime.datetime(2020, 1,1) )\
                                     & (time_sentitive_dataset["datetime"] < datetime.datetime(2020, 6,1))]

In [10]:

mean_values = train_df[time_sensitive_features].mean()
std_values = train_df[time_sensitive_features].std()

removable = (np.abs((train_df[time_sensitive_features] - mean_values) / std_values) < 3).all(axis=1)
if not removable.all():
    train_df_clean = train_df[removable]

removable = (np.abs((cv_df[time_sensitive_features] - mean_values) / std_values) < 3).all(axis=1)
if not removable.all():
    cv_df_clean = cv_df[removable]

removable = (np.abs((test_df[time_sensitive_features] - mean_values) / std_values) < 3).all(axis=1)
if not removable.all():
    test_df_clean = test_df[removable]

mean_values = train_df[time_sensitive_features].mean()
std_values = train_df[time_sensitive_features].std()

train_df_clean[time_sensitive_features] = (train_df_clean[time_sensitive_features] - mean_values) /std_values
cv_df_clean[time_sensitive_features] = (cv_df_clean[time_sensitive_features] - mean_values) / std_values
test_df_clean[time_sensitive_features] = (test_df_clean[time_sensitive_features] - mean_values) / std_values

/tmp/ipykernel_13712/3198831641.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_clean[time_sensitive_features] = (train_df_clean[time_sensitive_features] - mean_values) /std_values
/tmp/ipykernel_13712/3198831641.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv_df_clean[time_sensitive_features] = (cv_df_clean[time_sensitive_features] - mean_values) / std_values
/tmp/ipykernel_13712/3198831641.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [11]:
#timeseries_processing(cv_df_clean, time_sensitive_features, identifiers, labels, output, df_type = "cv")
print("Train")
print(train_df_clean.shape, train_df.shape)
print(train_df_clean["1-day-predict"].sum(), train_df["1-day-predict"].sum())

print("Cross val")
print(cv_df_clean.shape, cv_df.shape)
print(cv_df_clean["1-day-predict"].sum(), cv_df["1-day-predict"].sum())

print("Test")
print(test_df_clean.shape, test_df.shape)
print(test_df_clean["1-day-predict"].sum(), test_df["1-day-predict"].sum())


Train
(447687, 61) (613858, 61)
391 508
Cross val
(266863, 61) (335020, 61)
112 139
Test
(282252, 61) (405948, 61)
49 72


### Normalizing features

In [12]:
#q1 = train_df[time_sensitive_features].quantile(0.25)
#q3 = train_df[time_sensitive_features].quantile(0.75)
#iqr =  q3 - q1
#max_value = q3 + iqr*1.5
#min_value = q1 - iqr*1.5
#
#train_df = train_df[((train_df[time_sensitive_features] > min_value) & (train_df[time_sensitive_features] < max_value)).all(axis=1)]
#cv_df = cv_df[((cv_df[time_sensitive_features] > min_value) & (cv_df[time_sensitive_features] < max_value)).all(axis=1)]
#test_df = test_df[((test_df[time_sensitive_features] > min_value) & (test_df[time_sensitive_features] < max_value)).all(axis=1)]

#mean_values = train_df[time_sensitive_features].mean()
#std_values = train_df[time_sensitive_features].std()
#
#train_df.loc[:, time_sensitive_features] = (train_df[time_sensitive_features] - mean_values) / std_values
#
#cv_df.loc[:, time_sensitive_features] = (cv_df[time_sensitive_features] - mean_values) / std_values
#
#test_df.loc[:, time_sensitive_features] = (test_df[time_sensitive_features] - mean_values) / std_values

## Creating timeseries vectors for each dataset

In [13]:
timeseries_processing(cv_df_clean, time_sensitive_features, identifiers, labels, output, df_type = "cv")
timeseries_processing(test_df_clean, time_sensitive_features, identifiers, labels, output, df_type = "test")
timeseries_processing(train_df_clean, time_sensitive_features, identifiers, labels, output, df_type = "train")

UnboundLocalError: cannot access local variable 'rl_mlid_combos' where it is not associated with a value

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot(train_df, cv_df, test_df, prefix=""):
    figure_output =  pathlib.Path("figures")
    figure_output.mkdir(parents=True, exist_ok=True)
    for column in time_sensitive_features:
        fig = plt.figure()
        plot = sns.violinplot(data={"train":train_df[column], "cv":cv_df[column], "test":test_df[column]})
        #plot = sns.violinplot(data={"train":train_df[column]})
        plot.set(title=f"{prefix}_{column}")
        plot.get_figure().savefig(figure_output/f"{prefix}_{column}.png") 
        plt.close(fig)

In [ ]:
plot(train_df_clean, cv_df_clean, test_df_clean, "clean")
plot(train_df, cv_df, test_df, "raw")